In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import os
from datetime import timedelta



In [2]:
os.chdir('/Users/max/Deep_Sky/GitHub/Deep_Sky_Data_Science/viz')

In [3]:
df = pd.read_csv('../data/climate-sensitivity/ecs-timeline.csv', parse_dates=['date'])
df_filtered = df[['study', 'min', 'max', 'mean', 'date']]
df_filtered = df_filtered.dropna(subset=['min', 'max', 'mean'])
df_filtered = df_filtered.sort_values(by='date')

In [4]:
# Define the window size for the rolling average
window_size = 25  # Adjust this as needed

# Calculate the rolling averages
df_filtered['smoothed_min'] = df_filtered['min'].rolling(window_size, center=True).mean()
df_filtered['smoothed_max'] = df_filtered['max'].rolling(window_size, center=True).mean()

# Fill NaN values at the beginning and end of the DataFrame with the first and last valid rolling mean values, respectively
df_filtered['smoothed_min'].fillna(method='bfill', inplace=True)
df_filtered['smoothed_max'].fillna(method='bfill', inplace=True)
df_filtered['smoothed_min'].fillna(method='ffill', inplace=True)
df_filtered['smoothed_max'].fillna(method='ffill', inplace=True)

df_filtered.head()

,study,min,max,mean,date,smoothed_min,smoothed_max
0,Andronova and Schlesinger 2001,1.0,9.3,2.000000,2001-10-01,1.918,6.0356
1,Forest et al. 2002,1.4,7.7,3.500000,2002-01-04,1.918,6.0356
2,Kaufmann and Stern 2002,2.0,2.8,2.555556,2002-01-23,1.918,6.0356
3,Knutti et al. 2002,2.0,9.2,4.800000,2002-04-18,1.918,6.0356
4,IPCC TAR,1.7,4.2,3.000000,2002-06-14,1.918,6.0356


In [12]:
# Create a figure
from re import sub
from xarray import align
import datetime
import dash
import dash_core_components as dcc
import dash_html_components as html

add = dash.Dash(__name__)

# x-axis tick values
years = [2005, 2010, 2015, 2020]

fig = go.Figure()

# Add scatter plot for mean values
fig.add_trace(go.Scatter(
    x=df_filtered['date'],
    y=df_filtered['mean'],
    mode='markers',
    text=df_filtered['study'],  # Add study column to hover text
    hoverinfo='text',  # Show x, y, and text in hover info
    name='Mean',
    marker=dict(color='#ff574a'),
    showlegend=False
))

# Add shaded area between smoothed_min and smoothed_max lines
fig.add_trace(go.Scatter(
    x=df_filtered['date'],
    y=df_filtered['smoothed_min'],
    mode='lines',
    line=dict(width=0),
    showlegend=False,
    name='Smoothed Min',
    hoverinfo="none"
))

# Add shaded area between smoothed_min and smoothed_max lines
fig.add_trace(go.Scatter(
    x=df_filtered['date'],
    y=df_filtered['smoothed_max'],
    mode='lines',
    line=dict(width=0),
    fill='tonexty',
    fillcolor='rgba(255,0,0,0.2)',  # Light red color with 20% transparency
    showlegend=False,
    name='Smoothed Max',
    hoverinfo="none"
))

# Set the layout
fig.update_layout(
    title="UNCERTAINTY ON CLIMATE SENSITIVITY REMAINS SIGNIFICANT<br><sub>Scientists' Estimates of Equilibrium Climate Sensitivity (ECS) Over Time</sub>",
    #title = "LA SENSIBILITÉ DU CLIMAT À L'ÉQUILIBRE RESTE INCERTAINE<br><sub>Estimations des scientifiques à la sensibilité du climat à l'équilibre (SCE)",
    title_font={'family': 'Space Mono', 'size': 24, 'color': 'Black'},
    xaxis=dict(
        showgrid=False,  # Remove vertical gridlines
        range=[df_filtered['date'].min(), df_filtered['date'].max() + timedelta(days=365*5)],  # Restrict x-axis to earliest and latest data point
        tickmode='array',  # Use manually specified tick values
        tickvals=[datetime.datetime(year, 1, 1) for year in years],  # Set tick values to the start of each year
        ticktext=[str(year) for year in years]  # Set tick labels to the year strings

    ),
    yaxis=dict(
        gridcolor='gray',  # Turn horizontal gridlines black
        zeroline=False
    ),
    showlegend=False,
    plot_bgcolor='White',  
    paper_bgcolor='White',
    width =1000,
    height=500,
    font=dict(family="Space Mono", size = 18, color="Black")
)

# Add annotation for y = 4
fig.add_annotation(
    xref='x',
    yref='y', y=4.05,
    text="+4 WOULD BE<br>CATASTROPHIC", # . Currently,<br>1 in 10 Americans<br>live below sea line",
    x=df_filtered['date'].max() + timedelta(days=800), 
    #text="+4 SERAIT<br>CATASTROPHIQUE",
    #x=df_filtered['date'].max() + timedelta(days=900), 
    showarrow=False,
    font=dict(size=14, color='black'),
    align = 'left'
)

# Add annotation for y = 2
fig.add_annotation(
    xref='x',
    yref='y', 
    # english
    text="+2 IPCC TARGETED<br> THRESHOLD",
    y=2.05, x=df_filtered['date'].max() + timedelta(days=875), 
    
    # french
    #text="+2 SEUIL CIBLE<br>DU GIEC",
    #y=2.05, x=df_filtered['date'].max() + timedelta(days=800), 
    showarrow=False,
    font=dict(size=14, color='black'),
    align='left'
)

# Add a straight vertical red line at y=3.2
fig.add_shape(
    type="line",
    x0=df_filtered['date'].max(), 
    y0=2,
    x1=df_filtered['date'].max(), 
    y1=4.4,
    line=dict(
        color="red",
        width=3,
    ),
)

# Add annotation for deepskyclimate.com
fig.add_annotation(
    xref='paper', x=1, xanchor='right',
    yref='paper', y=-.07, yanchor='bottom',
    text="<b> \u2014\u2014\u2014 DEEPSKYCLIMATE.COM</b>",
    showarrow=False,
    font=dict(size=12, color='black')
)

# Add annotation for "°C" below the y-axis
fig.add_annotation(
    xref='paper', x=0, xanchor='left',
    yref='paper', y=-0.1, yanchor='bottom',
    text="°C",
    showarrow=False,
    font=dict(size=14, color='black')
)

# Add favicon
fig.add_layout_image(
    dict(
        #source="https://deep-sky-website-visualizations.s3.ca-central-1.amazonaws.com/DeepSky_favicon_light.png",  
        source="DeepSky_favicon_light.png",
        xref="paper", yref="paper",
        x=1, y=1.1,
        sizex=.16, sizey=.16,
        xanchor="right", yanchor="top",
        layer="above"
    )
)

fig.show()

In [13]:
import plotly.io as py

py.write_image(fig, '../figures/flawed_models/ecs_timeline.png')
py.write_html(fig, '../figures/flawed_models/ecs_timeline.html')

In [273]:
print(df_filtered['date'].max())

2022-09-19 00:00:00
